In [ ]:
import os
import time
import math
import random
import datetime as dt
from pathlib import Path
from typing import Optional, List, Dict
import yaml

import pandas as pd
from openai import OpenAI


# =========================
# Configuration
# =========================
MODEL_NAME = "gpt-4.1-mini"

MAX_CHARS_PER_CASE = 50000        # truncate full case text
MAX_CHARS_PER_SNIPPET = 12000     # truncate row text (snippet)
DELAY_BETWEEN_CALLS_SEC = 1.2     # anti rate-limit cushion

MAX_RETRIES = 5
RETRY_BASE_SLEEP = 1.5
RETRY_JITTER_MAX = 0.5

os.environ["OPENAI_API_KEY"] = ""

os.environ["INPUT_CSV"] 
os.environ["OUTPUT_DIR"]
os.environ["PROMPTS_YAML"]

input_csv = Path(os.environ["INPUT_CSV"])
output_dir = Path(os.environ["OUTPUT_DIR"])


# if you want logs
LOGS_DIR = output_dir / "logs"
LOGS_DIR.mkdir(parents=True, exist_ok=True)

# API key (prefer env var)
# os.environ["OPENAI_API_KEY"] = "sk-..."
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


PROMPTS_YAML = Path(os.environ["PROMPTS_YAML"])

_PROMPTS_CACHE: Dict[str, Any] = {}

In [92]:
# =========================
# Helpers
# =========================
def load_prompts_yaml(path: Path = PROMPTS_YAML) -> Dict[str, Any]:
    global _PROMPTS_CACHE
    if _PROMPTS_CACHE:
        return _PROMPTS_CACHE
    data = yaml.safe_load(path.read_text(encoding="utf-8"))
    if not isinstance(data, dict):
        raise ValueError("prompts.yaml must be a dict at top level")
    _PROMPTS_CACHE = data
    return _PROMPTS_CACHE

def _template_fields(tpl: str) -> set:
    return {fname for _, fname, _, _ in string.Formatter().parse(tpl) if fname}

def norm_text(x) -> str:
    if x is None:
        return ""
    if isinstance(x, float) and math.isnan(x):
        return ""
    return str(x)

def safe_int(x, default=0) -> int:
    try:
        if x is None:
            return default
        if isinstance(x, float) and math.isnan(x):
            return default
        return int(x)
    except Exception:
        return default

def build_case_fulltext_map(df: pd.DataFrame) -> Dict[str, str]:
    """
    case_file_name -> concatenation of all 'text' rows for that case
    """
    m: Dict[str, str] = {}
    for case, g in df.groupby("case_file_name", sort=False):
        parts = [norm_text(t).strip() for t in g["text"].tolist()]
        parts = [p for p in parts if p]
        full = "\n".join(parts)
        # normalize + truncate
        full = "\n".join([ln.strip() for ln in full.splitlines() if ln.strip()])
        m[str(case)] = full[:MAX_CHARS_PER_CASE]
    return m

def build_prompt_defendants(
    case_file_name: str,
    num_defendants: int,
    full_case_text: str,
    snippet_text: str,
) -> str:
    snippet_text = (snippet_text or "")[:MAX_CHARS_PER_SNIPPET]
    full_case_text = (full_case_text or "")[:MAX_CHARS_PER_CASE]

    prompts = load_prompts_yaml()
    cfg = prompts["defendants_labeling"]          # key in yaml
    tpl = cfg["user_template"]                    # the prompt template string

    return tpl.format(
        case_file_name=case_file_name,
        num_defendants=num_defendants,
        full_case_text=full_case_text,
        snippet_text=snippet_text,
    )

def parse_defendants_digits(output: str, num_defendants: int) -> List[int]:
    """
    Expect "0" or a digit string like "23" (for N<=9).
    Still robust if model returns "2 3" / "2,3" / "2-3" etc.
    """
    out = (output or "").strip()

    if not out:
        return []

    if out == "0":
        return []

    if out.isdigit():
        if num_defendants <= 9:
            defs = [int(ch) for ch in out]
        else:
            # N>9 is ambiguous in the "23" encoding; keep as one int (rare)
            defs = [int(out)]
    else:
        import re
        defs = [int(x) for x in re.findall(r"\d+", out)]

    # keep only 1..N, unique, sorted
    defs = sorted({d for d in defs if 1 <= d <= num_defendants})
    return defs
def format_defendants_str(defs: List[int]) -> str:
    # If none -> "0"
    if not defs:
        return "0"
    if any(d >= 10 for d in defs):
        return ",".join(str(d) for d in defs)
    return "".join(str(d) for d in defs)


def call_llm_with_retry(prompt: str) -> str:
    last_err: Optional[Exception] = None
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            resp = client.chat.completions.create(
                model=MODEL_NAME,
                messages=[
                    {"role": "system", "content": "You output ONLY digits. If none, output 0."},
                    {"role": "user", "content": prompt},
                ],
                temperature=0.0,
                max_tokens=8,
            )
            return (resp.choices[0].message.content or "").strip()
        except Exception as e:
            last_err = e
            sleep_s = RETRY_BASE_SLEEP * (2 ** (attempt - 1))
            sleep_s += random.random() * RETRY_JITTER_MAX
            time.sleep(sleep_s)
    raise last_err if last_err else RuntimeError("Unknown LLM error")


def build_local_context_map(df: pd.DataFrame, k: int = 3) -> Dict[int, str]:
    """
    Returns: row_index -> local_context (k rows before + current + k rows after) within same case_file_name
    Assumes df is in the same order you want (don’t sort unless you mean to).
    """
    ctx: Dict[int, str] = {}

    # keep original row indices
    for case, g in df.groupby("case_file_name", sort=False):
        idxs = g.index.tolist()
        texts = [norm_text(t).strip() for t in g["text"].tolist()]

        for pos, row_idx in enumerate(idxs):
            lo = max(0, pos - k)
            hi = min(len(idxs), pos + k + 1)

            lines = []
            for j in range(lo, hi):
                tag = "CURRENT" if j == pos else ("PREV" if j < pos else "NEXT")
                lines.append(f"{tag}: {texts[j]}")

            local = "\n".join([ln for ln in lines if ln.strip()])
            ctx[row_idx] = local[:MAX_CHARS_PER_SNIPPET]  # keep it bounded
    return ctx


In [93]:
def run_defendants_labeling(
    input_csv: Path,
    output_dir: Path,
) -> Path:
    ts = dt.datetime.now().strftime("%Y%m%d_%H%M%S")
    out_csv = output_dir / f"defendants_pred_{ts}.csv"
    log_path = LOGS_DIR / f"run_defendants_{ts}.log"

    df = pd.read_csv(input_csv)

    # make sure pred exists (we do NOT touch defendants_str - it's the label)
    if "pred" not in df.columns:
        df["pred"] = ""

    required = {"case_file_name", "text", "defendants_str", "pred", "num_defendants"}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"CSV missing required columns: {sorted(missing)}")

    # enforce output schema
    df = df[["case_file_name", "text", "defendants_str", "pred", "num_defendants"]]

    case_map = build_case_fulltext_map(df)

    with open(log_path, "w", encoding="utf-8") as log_f:
        total = len(df)
        log_f.write(f"[{ts}] Start defendants prediction: {total} rows\n")
        log_f.flush()

        for idx, row in df.iterrows():
            case = str(row.get("case_file_name", "")).strip()
            n = safe_int(row.get("num_defendants"), 0)
            snippet = norm_text(row.get("text", "")).strip()

            # always predict for ALL rows
            if n <= 0 or not snippet:
                df.at[idx, "pred"] = "0"
                log_f.write(f"[{idx+1}/{total}] case={case} -> pred='0' (empty text or num_defendants={row.get('num_defendants')})\n")
                log_f.flush()
                continue

            full_case_text = case_map.get(case, "")
            prompt = build_prompt_defendants(case, n, full_case_text, snippet)

            try:
                raw = call_llm_with_retry(prompt)   # should be digits / 0
                df.at[idx, "pred"] = raw            # keep prediction only in pred

                # (optional) normalize outputs like "0.0" -> "0"
                s = norm_text(raw).strip()
                if s.endswith(".0"):
                    s = s[:-2]
                    df.at[idx, "pred"] = s

                log_f.write(f"[{idx+1}/{total}] case={case} -> pred='{df.at[idx,'pred']}'\n")
                log_f.flush()
                time.sleep(DELAY_BETWEEN_CALLS_SEC)

            except Exception as e:
                df.at[idx, "pred"] = "0"
                log_f.write(f"[{idx+1}/{total}] case={case} -> ERROR: {e} (set pred=0)\n")
                log_f.flush()

        log_f.write(f"[{dt.datetime.now().strftime('%Y%m%d_%H%M%S')}] Finished.\n")
        log_f.flush()

    df.to_csv(out_csv, index=False)
    print(f"✅ saved: {out_csv}")
    print(f"🗒️  log:   {log_path}")
    return out_csv

In [ ]:
run_defendants_labeling(
    input_csv=input_csv,
    output_dir=output_dir
)

In [95]:
from pathlib import Path
from typing import Union, Set, Any
import pandas as pd
import numpy as np
import re

def evaluate_defendants_csv(csv_path: Union[str, Path]) -> pd.DataFrame:
    """
    Input CSV columns:
      case_file_name,text,defendants_str,pred,num_defendants

    defendants_str = true value ("0" means none)
    pred           = predicted value ("0" means none)

    Returns a df with rows: overall, N=2, N=3
    Columns: micro_precision, micro_recall, micro_f1, avg_jaccard, accuracy
    (accuracy here = exact set match, per-row)
    """

    df = pd.read_csv(csv_path)

    required = {"defendants_str", "pred", "num_defendants"}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"Missing required columns: {sorted(missing)}")

    def _to_int(x, default=0) -> int:
        try:
            if x is None:
                return default
            if isinstance(x, float) and np.isnan(x):
                return default
            return int(float(x))
        except Exception:
            return default

    def _clean_str(x: Any) -> str:
        if x is None:
            return "0"
        if isinstance(x, float) and np.isnan(x):
            return "0"
        s = str(x).strip()
        if s == "":
            return "0"
        # normalize "23.0" -> "23"
        if re.fullmatch(r"\d+\.0", s):
            s = s[:-2]
        return s

    def _parse_set(val: Any, n: int) -> Set[int]:
        s = _clean_str(val)
        if s == "0":
            return set()

        if not s.isdigit():
            nums = [int(x) for x in re.findall(r"\d+", s)]
            return {d for d in nums if 1 <= d <= n}

        if n <= 9:
            return {int(ch) for ch in s if ch.isdigit() and 1 <= int(ch) <= n}

        nums = [int(x) for x in re.findall(r"\d+", s)]
        return {d for d in nums if 1 <= d <= n}

    def _score(sub: pd.DataFrame) -> dict:
        TP = FP = FN = 0
        jaccs = []
        exact_matches = 0
        counted = 0

        for _, r in sub.iterrows():
            n = _to_int(r["num_defendants"], 0)
            if n <= 0:
                continue

            t = _parse_set(r["defendants_str"], n)
            p = _parse_set(r["pred"], n)

            TP += len(t & p)
            FP += len(p - t)
            FN += len(t - p)

            union = len(t | p)
            jaccs.append(1.0 if union == 0 else len(t & p) / union)

            exact_matches += int(t == p)
            counted += 1

        prec = TP / (TP + FP) if (TP + FP) else 1.0
        rec  = TP / (TP + FN) if (TP + FN) else 1.0
        f1   = (2 * prec * rec / (prec + rec)) if (prec + rec) else 0.0
        avg_j = float(np.mean(jaccs)) if jaccs else 0.0
        acc = (exact_matches / counted) if counted else 0.0

        return {
            "micro_precision": prec,
            "micro_recall": rec,
            "micro_f1": f1,
            "accuracy": acc,
            "avg_jaccard": avg_j
        }

    df["num_defendants_int"] = df["num_defendants"].apply(_to_int)

    overall = _score(df)
    n2 = _score(df[df["num_defendants_int"] == 2])
    n3 = _score(df[df["num_defendants_int"] == 3])

    out = pd.DataFrame([overall, n2, n3], index=["overall", "N=2", "N=3"])
    return out

In [ ]:
file  = ""
eval_df = evaluate_defendants_csv(file)
print(eval_df)

         micro_precision  micro_recall  micro_f1  accuracy  avg_jaccard
overall         0.749917      0.787478  0.768239  0.719039     0.739921
N=2             0.812620      0.826848  0.819672  0.761682     0.781153
N=3             0.714837      0.761010  0.737201  0.690094     0.710293


In [ ]:
results_dir = output_dir / "results"
results_dir.mkdir(parents=True, exist_ok=True)

fn = results_dir / f"eval_defendants_{dt.datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
eval_df.to_csv(fn, index=True, encoding="utf-8-sig")

print(f"Saved evaluation CSV: {fn}")